In [1]:
# === Quick responsiveness test inside a notebook ===
# Prereqs:
#   pip install ollama pandas
from pathlib import Path
import json, re
import pandas as pd
from ollama import chat   # matches your example
# ---------------- Configuration ----------------
CASE_BACKGROUND = """
This is a hypothetical antitrust investigation about widget pricing.
Documents are responsive if they discuss widget pricing strategy,
communications with competitors, or evidence of market allocation.
"""
FOLDER = "./test_docs"  # folder containing files to test
MODELS = ["qwen3:4b","gemma3n:e4b",  "llama3.2:3b"]
SYSTEM_PROMPT = (
   "You are an eDiscovery reviewer. "
   "Decide if the document is RESPONSIVE to the case background. "
   "Output JSON: {\"label\":\"Responsive|Not Responsive|Needs Review\", \"rationale\":\"...\"}"
)
USER_TEMPLATE = "CASE BACKGROUND:\n{case}\n\nDOCUMENT:\n{doc}"
def read_file_text(p: Path, max_chars=5000) -> str:
   try:
       return p.read_text(encoding="utf-8", errors="ignore")[:max_chars]
   except:
       return ""
def call_model(model: str, case_text: str, doc_text: str) -> dict:
   msg = USER_TEMPLATE.format(case=case_text, doc=doc_text)
   resp = chat(
       model=model,
       messages=[
           {"role": "system", "content": SYSTEM_PROMPT},
           {"role": "user", "content": msg},
       ],
       think=False
   )
   raw = resp.message.content
   m = re.search(r"\{.*\}", raw, re.DOTALL)
   try:
       data = json.loads(m.group(0) if m else raw)
       label = str(data.get("label","Needs Review")).strip()
       rationale = str(data.get("rationale","")).strip()
   except Exception:
       label, rationale = "Needs Review", f"Parse error: {raw[:100]}"
   return {"label": label, "rationale": rationale}
def majority_vote(labels):
   votes = {"Responsive":0, "Not Responsive":0, "Needs Review":0}
   for l in labels:
       if l in votes: votes[l]+=1
       else: votes["Needs Review"]+=1
   best = max(votes.items(), key=lambda kv: kv[1])
   # tie -> Needs Review
   if list(votes.values()).count(best[1]) > 1:
       return "Needs Review"
   return best[0]
results = []
for f in Path(FOLDER).glob("*"):
   print(f)
   if not f.is_file(): continue
   text = read_file_text(f)
   model_votes, model_rats = {}, {}
   for m in MODELS:
       out = call_model(m, CASE_BACKGROUND, text)
       model_votes[m] = out["label"]
       model_rats[m] = out["rationale"]
   final = majority_vote(list(model_votes.values()))
   print(final)
   results.append({
       "file": f.name,
       "final_vote": final,
       **{f"vote_{m}": v for m,v in model_votes.items()},
       **{f"rat_{m}": r for m,r in model_rats.items()},
   })
df = pd.DataFrame(results)
df

test_docs\New Text Document (2).txt
Not Responsive
test_docs\New Text Document (3).txt
Needs Review
test_docs\New Text Document (4).txt
Not Responsive
test_docs\New Text Document.txt
Responsive


,file,final_vote,vote_qwen3:4b,vote_gemma3n:e4b,vote_llama3.2:3b,rat_qwen3:4b,rat_gemma3n:e4b,rat_llama3.2:3b
0,New Text Document (2).txt,Not Responsive,Needs Review,Not Responsive,Not Responsive,Parse error: <think>\nWe are given a case back...,This document is an invitation to a holiday pa...,The document does not discuss widget pricing s...
1,New Text Document (3).txt,Needs Review,Needs Review,Responsive,Not Responsive,Parse error: <think>\nWe are given a case back...,"The document explicitly discusses ""Regional Ma...",The document does not discuss widget pricing s...
2,New Text Document (4).txt,Not Responsive,Needs Review,Not Responsive,Not Responsive,Parse error: <think>\nWe are given a case back...,"The document discusses employee spotlights, vo...",The document does not mention widget pricing s...
3,New Text Document.txt,Responsive,Needs Review,Responsive,Responsive,Parse error: <think>\nWe are given a case back...,The document directly discusses widget pricing...,The document discusses widget pricing strategy...


In [6]:
import ollama
print(ollama.list())

models=[Model(model='qwen3:4b', modified_at=datetime.datetime(2025, 9, 9, 18, 50, 57, 458398, tzinfo=TzInfo(+08:00)), digest='e55aed6fe643f9368b2f48f8aaa56ec787b75765da69f794c0a0c23bfe7c64b2', size=2497293918, details=ModelDetails(parent_model='', format='gguf', family='qwen3', families=['qwen3'], parameter_size='4.0B', quantization_level='Q4_K_M')), Model(model='llama3.2:3b', modified_at=datetime.datetime(2025, 9, 5, 15, 41, 29, 122393, tzinfo=TzInfo(+08:00)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='3.2B', quantization_level='Q4_K_M')), Model(model='gemma3n:e4b', modified_at=datetime.datetime(2025, 9, 4, 18, 53, 5, 88870, tzinfo=TzInfo(+08:00)), digest='15cb39fd9394fd2549f6df9081cfc84dd134ecf2c9c5be911e5629920489ac32', size=7547589116, details=ModelDetails(parent_model='', format='gguf', family='gemma3n', families=['gemma3n'], par